In [1]:
# IMPORTS
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog, leaguegamefinder, leaguegamelog, commonteamroster, teamdashlineups, commonplayerinfo
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint

In [2]:
target_player_name = "Tyrese Maxey"
# Get the list of all NBA players
nba_players = players.get_players()

# Find the player with the specified name
target_player = next(player for player in nba_players if player['full_name'] == target_player_name)
# {'id': 203954, 'full_name': 'Joel Embiid', 'first_name': 'Joel', 'last_name': 'Embiid', 'is_active': True}

# Extract the relevant data for the player's career
career_stats = playercareerstats.PlayerCareerStats(player_id=target_player['id'])
career_per_season_data = career_stats.get_data_frames()[0]
    
# Create a DataFrame for the player's career stats
player_career_stats_df = pd.DataFrame(career_per_season_data)
    
# Create a DataFrame with player information
nba_players_df = pd.DataFrame(nba_players)
    
# Player will be same as above
game_log = playergamelog.PlayerGameLog(player_id=target_player['id']) 
game_log_data = game_log.get_data_frames()[0]

# Create a DataFrame for the player's career stats
game_log_df = pd.DataFrame(game_log_data)

# Merge the two DataFrames based on 'PLAYER_ID'
game_log_df = pd.merge(game_log_df, nba_players_df[['id', 'full_name']], left_on='Player_ID', right_on='id', how='left')

# Drop the redundant 'id' column
game_log_df = game_log_df.drop(columns=['id'])

# Reorder the columns to make 'full_name' the second column and rename  to 'FULL_NAME'
game_log_df = game_log_df[['Player_ID', 'full_name'] + [col for col in game_log_df.columns if col not in ['Player_ID', 'full_name']]].rename(columns={'full_name': 'FULL_NAME'})

game_log_df

,Player_ID,FULL_NAME,SEASON_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,1630178,Tyrese Maxey,22023,0022300255,"NOV 27, 2023",PHI vs. LAL,W,36,9,20,...,3,3,8,1,0,0,2,31,25,1
1,1630178,Tyrese Maxey,22023,0022300239,"NOV 25, 2023",PHI @ OKC,W,41,8,17,...,7,8,3,0,1,1,5,28,6,1
2,1630178,Tyrese Maxey,22023,0022300232,"NOV 22, 2023",PHI @ MIN,L,34,7,19,...,3,5,8,0,1,3,0,16,-18,1
3,1630178,Tyrese Maxey,22023,0022300040,"NOV 21, 2023",PHI vs. CLE,L,47,11,28,...,4,4,6,1,0,1,2,30,-8,1
4,1630178,Tyrese Maxey,22023,0022300208,"NOV 19, 2023",PHI @ BKN,W,36,8,20,...,1,2,10,0,0,1,0,25,21,1
5,1630178,Tyrese Maxey,22023,0022300029,"NOV 17, 2023",PHI @ ATL,W,40,5,13,...,4,4,8,2,0,2,1,19,1,1
6,1630178,Tyrese Maxey,22023,0022300194,"NOV 15, 2023",PHI vs. BOS,L,39,6,15,...,3,3,4,0,0,2,3,20,9,1
7,1630178,Tyrese Maxey,22023,0022300019,"NOV 14, 2023",PHI vs. IND,L,38,9,23,...,4,4,5,0,1,4,2,27,-6,1
8,1630178,Tyrese Maxey,22023,0022300180,"NOV 12, 2023",PHI vs. IND,W,39,20,32,...,4,7,5,0,3,2,2,50,17,1
9,1630178,Tyrese Maxey,22023,0022300008,"NOV 10, 2023",PHI @ DET,W,39,9,19,...,6,6,11,3,1,0,3,29,17,1


In [3]:
import pandas as pd

def calculate_player_averages(target_player_name):
    # Get the list of all NBA players
    nba_players = players.get_players()

    # Find the player with the specified name
    target_player = next(player for player in nba_players if player['full_name'] == target_player_name)

    # Extract the relevant data for the player's career
    career_stats = playercareerstats.PlayerCareerStats(player_id=target_player['id'])
    career_per_season_data = career_stats.get_data_frames()[0]

    # Create a DataFrame for the player's career stats
    player_career_stats_df = pd.DataFrame(career_per_season_data)

    # Create a DataFrame with player information
    nba_players_df = pd.DataFrame(nba_players)

    # Get the player's game log for the current season
    game_log = playergamelog.PlayerGameLog(player_id=target_player['id'], season="2023-24")
    game_log_data = game_log.get_data_frames()[0]

    # Create a DataFrame for the player's game log
    game_log_df = pd.DataFrame(game_log_data)
    game_log_df['PLAYER_NAME'] = target_player_name

    # Calculate average points per game (PPG), average rebounds per game (RPG),
    # average assists per game (APG), and average minutes per game (MPG)
    average_stats = game_log_df.groupby('PLAYER_NAME').agg({
        'PTS': 'mean',
        'REB': 'mean',
        'AST': 'mean',
        'MIN': 'mean'
    }).reset_index()

    return average_stats

# Example usage
target_player_name = "Tyrese Maxey"
average_stats = calculate_player_averages(target_player_name)
average_stats

,PLAYER_NAME,PTS,REB,AST,MIN
0,Tyrese Maxey,26.647059,4.823529,6.882353,38.352941


In [75]:
# Replace 'path_to_file' with the actual path to your CSV file
file_path = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/PHI_roster.csv'

# Read the CSV file into a DataFrame
phi_df = pd.read_csv(file_path)

# Display the DataFrame
phi_df

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612755,2023,0,Tyrese Maxey,Tyrese,tyrese-maxey,0,G,6-2,200,"NOV 04, 2000",23.0,3,Kentucky,1630178,#21 Pick in 2020 Draft
1,1610612755,2023,0,KJ Martin,KJ,kj-martin,1,F,6-6,215,"JAN 06, 2001",22.0,3,IMG Academy (FL),1630231,Traded from LAC on 11/01/23
2,1610612755,2023,0,Marcus Morris Sr.,Marcus,marcus-morris-sr,5,F,6-8,218,"SEP 02, 1989",34.0,12,Kansas,202694,Traded from LAC on 11/01/23
3,1610612755,2023,0,Mo Bamba,Mo,mo-bamba,7,C,7-0,231,"MAY 12, 1998",25.0,5,Texas,1628964,Signed on 07/09/23
4,1610612755,2023,0,De'Anthony Melton,De'Anthony,deanthony-melton,8,G,6-3,200,"MAY 28, 1998",25.0,5,Southern California,1629001,Traded from MEM on 06/24/22
5,1610612755,2023,0,Kelly Oubre Jr.,Kelly,kelly-oubre-jr,9,F-G,6-7,203,"DEC 09, 1995",27.0,8,Kansas,1626162,Signed on 09/26/23
6,1610612755,2023,0,Jaden Springer,Jaden,jaden-springer,11,G,6-4,202,"SEP 25, 2002",21.0,2,Tennessee,1630531,#28 Pick in 2021 Draft
7,1610612755,2023,0,Tobias Harris,Tobias,tobias-harris,12,F,6-8,226,"JUL 15, 1992",31.0,12,Tennessee,202699,Traded from LAC on 02/06/19
8,1610612755,2023,0,Ricky Council IV,Ricky,ricky-council-iv,16,G,6-6,207,"AUG 03, 2001",22.0,R,Arkansas,1641741,Signed on 07/01/23
9,1610612755,2023,0,Joel Embiid,Joel,joel-embiid,21,C-F,7-0,280,"MAR 16, 1994",29.0,7,Kansas,203954,#3 Pick in 2014 Draft


In [76]:
# Apply the function to each player in phi_df
phi_stats_list = []

for index, row in phi_df.iterrows():
    player_name = row['PLAYER']
    player_stats = calculate_player_averages(player_name)
    phi_stats_list.append(player_stats)

# Combine the individual player stats into a single DataFrame
phi_stats_df = pd.concat(phi_stats_list, ignore_index=True)

phi_stats_df['TeamID'] = phi_df['TeamID']
# Display the DataFrame with average stats for each player
phi_stats_df

,PLAYER_NAME,PTS,REB,AST,MIN,TeamID
0,Tyrese Maxey,26.647059,4.823529,6.882353,38.352941,1610612755
1,KJ Martin,1.555556,0.888889,0.222222,6.111111,1610612755
2,Marcus Morris Sr.,4.000000,1.500000,0.300000,10.000000,1610612755
3,Mo Bamba,2.571429,1.571429,0.000000,3.285714,1610612755
4,De'Anthony Melton,12.117647,4.058824,3.647059,29.823529,1610612755
5,Kelly Oubre Jr.,16.250000,5.125000,0.625000,29.000000,1610612755
6,Jaden Springer,3.181818,2.000000,0.909091,11.636364,1610612755
7,Tobias Harris,18.588235,6.235294,2.705882,35.411765,1610612755
8,Joel Embiid,32.000000,11.250000,6.625000,34.500000,1610612755
9,Patrick Beverley,2.882353,3.058824,2.000000,16.823529,1610612755


In [25]:
phi_stats_df.to_csv('PHI_player_stats.csv', index=False)

In [5]:
# Get the list of all NBA teams
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)
# Rename the 'id' column to 'teamID'
teams_df.rename(columns={'id': 'TeamID'}, inplace=True)

# GET SPECIFIC TEAMS ROSTER
team_abbreviation = 'NYK'
team_id = teams_df.loc[teams_df['abbreviation'] == team_abbreviation, 'TeamID'].iloc[0]
print(f"The team ID for {team_abbreviation} is {team_id}.")
teamfinder = commonteamroster.CommonTeamRoster(team_id=team_id, league_id_nullable='00')
teams = teamfinder.get_data_frames()[0]
team_id = teams['TeamID'].unique().tolist()
team_ids = teams['TeamID'].unique().tolist()
teams_roster = []

for team_id in team_ids:
    player_stats_data = commonteamroster.CommonTeamRoster(season='2023-24', team_id=team_id)
    df = player_stats_data.common_team_roster.get_data_frame()
    
    # Add the TeamAbbreviation column
    df['TeamAbbreviation'] = team_abbreviation
    
    teams_roster.append(df)

# Concatenate all the DataFrames in teams_roster list
df = pd.concat(teams_roster, ignore_index=True)
df

The team ID for NYK is 1610612752.


,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED,TeamAbbreviation
0,1610612752,2023,00,Jaylen Martin,Jaylen,jaylen-martin,None,G,6-6,216,"JAN 28, 2004",19.0,R,,1641798,Signed on 07/03/23,NYK
1,1610612752,2023,00,Donte DiVincenzo,Donte,donte-divincenzo,0,G,6-4,203,"JAN 31, 1997",26.0,5,Villanova,1628978,Signed on 07/08/23,NYK
2,1610612752,2023,00,Jacob Toppin,Jacob,jacob-toppin,00,F,6-8,200,"MAY 08, 2000",23.0,R,Kentucky,1631210,Signed on 07/06/23,NYK
3,1610612752,2023,00,Miles McBride,Miles,miles-mcbride,2,G,6-1,195,"SEP 08, 2000",23.0,2,West Virginia,1630540,Draft Rights Traded from OKC on 07/30/21,NYK
4,1610612752,2023,00,Josh Hart,Josh,josh-hart,3,G,6-4,215,"MAR 06, 1995",28.0,6,Villanova,1628404,Traded from POR on 02/09/23,NYK
5,1610612752,2023,00,Immanuel Quickley,Immanuel,immanuel-quickley,5,G,6-2,190,"JUN 17, 1999",24.0,3,Kentucky,1630193,Draft Rights Traded from OKC on 11/20/20,NYK
6,1610612752,2023,00,Quentin Grimes,Quentin,quentin-grimes,6,G,6-5,210,"MAY 08, 2000",23.0,2,Houston,1629656,Draft Rights Traded from LAC on 07/30/21,NYK
7,1610612752,2023,00,DaQuan Jeffries,DaQuan,daquan-jeffries,8,G-F,6-5,222,"AUG 30, 1997",26.0,4,Tulsa,1629610,Signed on 09/14/22,NYK
8,1610612752,2023,00,RJ Barrett,RJ,rj-barrett,9,F-G,6-6,214,"JUN 14, 2000",23.0,4,Duke,1629628,#3 Pick in 2019 Draft,NYK
9,1610612752,2023,00,Jalen Brunson,Jalen,jalen-brunson,11,G,6-2,190,"AUG 31, 1996",27.0,5,Villanova,1628973,Signed on 07/12/22,NYK


In [78]:
# Apply the function to each player in phi_df
team_stats_list = []

for index, row in df.iterrows():
    player_name = row['PLAYER']
    player_stats = calculate_player_averages(player_name)

    # Check if player_stats is not None (i.e., player found)
    if player_stats is not None:
        team_stats_list.append(player_stats)

# Combine the individual player stats into a single DataFrame
team_stats_df = pd.concat(team_stats_list, ignore_index=True)

# Add TeamID and team_abbv columns to the final DataFrame
team_stats_df['TeamID'] = df['TeamID']
team_stats_df['team_abbv'] = df['TeamAbbreviation']

# Display the DataFrame with average stats for each player
team_stats_df


StopIteration: 

In [61]:
team_stats_df.to_csv('OKC_player_stats.csv', index=False)

In [8]:
import pandas as pd
OKC_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/OKC_player_stats.csv'
DEN_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/DEN_player_stats.csv'
PHI_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/PHI_player_stats.csv'
MIL_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/MIL_player_stats.csv'
MIN_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/MIN_player_stats.csv'
ORL_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/ORL_player_stats.csv'
HOU_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/HOU_player_stats.csv'
LAL_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/LAL_player_stats.csv'
BOS_player_stats = 'C:/Users/fishm/Documents/Sports Data/NBA API/Data/BOS_player_stats.csv'
# List of file names
file_names = [OKC_player_stats, DEN_player_stats, PHI_player_stats, MIL_player_stats, MIN_player_stats,
              ORL_player_stats, HOU_player_stats, LAL_player_stats, BOS_player_stats]

# Create an empty list to store DataFrames
dfs = []

# Read each CSV file and append its DataFrame to the list
for file_name in file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
concatenated_df = pd.concat(dfs, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
#concatenated_df.to_csv('combined_player_stats.csv', index=False)